# Understanding E-BFMI

When I was studying a model for my disseration, a warning was displayed by Stan: E-BFMI was too low. In order to understand why it happens, [this link](https://discourse.mc-stan.org/t/issues-with-e-bfmi-missing-doc-confusing-name-etc/22553) is very useful. Moreover, [this article](https://arxiv.org/pdf/1604.00695.pdf) is very explanatory, so I will follow it in this notebook.  

Let $\pi$ be the target distribution that admits a density with respect to the Lebesgue measure. Markov chain Monte Carlo samples $x^{(1)}, x^{(2)}, \dots, x^{(n)}$ from a Markov chain that converges to the target distribution. This is a consequence of a result of convergence in Markov chains theory. In the Metropolis-Hastings algorithm, for instance, given $X_i = x$, we sample $Y$ from $P(Y = y \mid X = x) = Q(y|x)$, a propose mechanism. With probability $A(y|x)$, we accept $y$, otherwise we reject it and set $X_{i+1} = X_i$. Eventually, the Markov chain will explore the entire target distribution. The original Markov chain Monte Carlo algorithm, and one still commonly in use today, utilizes a Gaussian distribution as its proposal mechanism. 

## Hamiltonian Monte Carlo 

We follow the [this text](https://arxiv.org/pdf/1701.02434.pdf). This method was developed in the late 1980s as Hybrid Monte Carlo to tackle calculations in Lattice Quantum Chromodynamics. Instead of moving in the parameter space randomly with uninformed jumps, the direction from the vector field given by the gradients are used to trace out a trajectory through the *typical set*, the region which has significant contribution to the expectations. However, if only the gradient was used, the trajectory would pull towards the mode of the distribution, so more geometric constraints are needed. In order to a satellite rotate around the Earth, we have to endow ir with enough momentum to counteract the gravitational field, turning the system into a conservative one. 

First, we introduce auxiliary momentum parameters $p_n$ (lift) of the same dimension from the parameter space $\Omega \subseteq \mathbb{R}^D$. Then $q_n$ turns to $(q_n, p_n)$, with the use the joint probability distribution $\pi(q,p) = \pi(p\mid q)\pi(q)$. Particularly, we use 

$$
\pi(q,p) = e^{-H(q,p)}, 
$$

such that $H$ is the *Hamiltonian*. Note that $H(q,p) = -\log \pi(p\mid q) - \log \pi(q) =: K(p,q) + V(q)$. We call $K$ the kinetic energy, and $V$ the potential energy. The vector field is generated by Hamilton's equations, 

$$
\frac{dq}{dt} = \frac{\partial H}{\partial p} = \frac{\partial K}{\partial p}
$$
$$
\frac{dp}{dt} = -\frac{\partial H}{\partial q} = -\frac{\partial K}{\partial q} - \frac{d V}{d q}.
$$

Therefore, we are able to define the Hamiltonian flows $\phi_t : (p,q) \to (p,q), \forall t \in \mathbb{R}$.

## Diagnosing Suboptimal Cotangent Desintegrations 

The cotangent desintegration is the conditional distribution over the aumentation of the parameter space.  We want to quantity the efficacy of the momentum resampling. Let $H(p,q) = E$ be the energy of the system and $\pi_{E|q}$ the distribution of $E$ induced by a momentum resampling at position $q$. The closer this this distribution is to $\pi_E$, the faster the random walk will explore the energies. The *momentum resampling* is the composition of a projection to the sample space and random lift stage. We would like that 

$$
\log \frac{d\pi_{E|q}}{d\pi_{E}} = 0, \forall q \in Q.
$$

Then we use $\mathbb{E}_{\pi}\left[\log \frac{d\pi_{E|q}}{d\pi_{E}}\right]$. One theoretically appealing choice is the Bayesian fraction of missing information

$$
BFMI = \frac{\mathbb{E}_{\pi}[Var_{\pi_{E|q}}[E\mid q]]}{Var_{\pi_{E}}[E]},
$$

which quantifies how insufficient the energy variation induced by the momentum resampling is. The basic ideia is that if the jumps between the slices of the Hamiltonian (the level sets) can be short and need more momentum.

## Hierarchical Target Example 

We will reproduce the example given in the article regarding the **eight schools posterior distribution**. It comes from [Sturtz,  Uwe Ligges, Andrew Gelman](https://www.jstatsoft.org/article/view/v012i03/v12i03.pdf)

$$
y_n \sim \operatorname{Normal}(\theta_n, \sigma_n^2), 
$$

such that 

$$
\theta_n \sim \operatorname{Normal}(\mu, \tau^2) \\
\mu \sim \operatorname{Normal}(0,10) \\
\tau \sim \operatorname{Half-C}(0,10), \\
$$

with $y_n, \sigma_n$ being the data. 